In [1]:
#Other imports
import operator
%matplotlib inline
from collections import namedtuple
from sklearn.linear_model import LogisticRegression
from __future__ import print_function, division
import pandas as pd
import time
import os
import copy
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

#Torch imports
import torchvision.models as models
from torchvision import transforms
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch import optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import torch.optim as optim

#### Alexnet Data

In [2]:
#Getting alexnet
alexnet = models.alexnet(pretrained=True)

#Basic transforms using mean and std
# Transform variable to convert images to the right size
transform = transforms.Compose([            
 transforms.Resize(256),                   
 transforms.CenterCrop(224),              
 transforms.ToTensor(),                    
 transforms.Normalize(                     
 mean=[0.485, 0.456, 0.406],             
 std=[0.229, 0.224, 0.225]                  
 )])

#### Downloading and setting up CIFAR-10 File

In [3]:
# Getting datafile, creating directory
datafile = CIFAR10(root='datafile/', download=True, transform=transform)
test_datafile = CIFAR10(root='datafile/', train=False, transform=transform)

#Getting 10 classes from the datafile
classes = datafile.classes

torch.manual_seed(43)
valsize = 10000
trainsize = len(datafile) - valsize

trainds, valds = random_split(datafile, [trainsize, valsize])

batchsize=100

train_loader = DataLoader(trainds, batchsize, shuffle=False, num_workers=8, pin_memory=True)
val_loader = DataLoader(valds, batchsize, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_datafile, batchsize, num_workers=8, pin_memory=True)

Files already downloaded and verified


## Part B of Assignment

In [4]:
#Getting 1000 classes from imagenet classes 
#Source: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a 
with open("imagenet_classes.txt") as f:
    classes = eval(f.read())

# Creating temp collections
holder = []
dic = {}
current = ''

#Iterating to batch of testloader and gets the output from alexnet
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        out = alexnet(images) 


        for j in range(0,batchsize):
            sorted_out, indices = torch.sort(out,descending=True)
            percentage = F.softmax(out,dim=1)[j]*100
            results = [(classes[i.item()],percentage[i].item()) for i in indices[j][:5]]
            holder.append(results[0][0])

# Setting classification names
holder.sort()

#Getting the dictionary populated 
for z in holder:
    if current != z:
        count = 1
        dic[z] = count
        current = z
    else:
        count = count + 1
        dic[z] = count 
        current = z
        
#sorting the dictionary 
sorteddictionary = dict( sorted(dic.items(), key=operator.itemgetter(1),reverse=True))

#Printing top 10 classifications 
dicMax = 1
for i in sorteddictionary:
    print(dicMax,') ',i,':',sorteddictionary[i])
    dicMax = dicMax + 1
    if(dicMax > 10):
        break

1 )  moving van : 1065
2 )  fox squirrel, eastern fox squirrel, Sciurus niger : 627
3 )  sorrel : 331
4 )  container ship, containership, container vessel : 285
5 )  English foxhound : 259
6 )  milk can : 232
7 )  Japanese spaniel : 224
8 )  thresher, thrasher, threshing machine : 224
9 )  Dandie Dinmont, Dandie Dinmont terrier : 220
10 )  chain saw, chainsaw : 171


In [5]:
#Short forms from CIFAR10
cyactual = pd.Series(['AP','AM','B','C','DR','DG','FG','H','S','T'], name='CIFAR-10')

#Short forms from above 
cypredicted = pd.Series(['MV','FESS','SO','CCCV','EF','MC','JS','TTTM','DDDDT','CSC'], name='AlexNet')
dfconfusion = pd.crosstab(cyactual, cypredicted)

#Printing matrix
print(dfconfusion)

AlexNet   CCCV  CSC  DDDDT  EF  FESS  JS  MC  MV  SO  TTTM
CIFAR-10                                                  
AM           0    0      0   0     1   0   0   0   0     0
AP           0    0      0   0     0   0   0   1   0     0
B            0    0      0   0     0   0   0   0   1     0
C            1    0      0   0     0   0   0   0   0     0
DG           0    0      0   0     0   0   1   0   0     0
DR           0    0      0   1     0   0   0   0   0     0
FG           0    0      0   0     0   1   0   0   0     0
H            0    0      0   0     0   0   0   0   0     1
S            0    0      1   0     0   0   0   0   0     0
T            0    1      0   0     0   0   0   0   0     0


## Part C of Assignment

In [6]:
device = ("cuda:0" if torch.cuda.is_available() else "cpu")


model = models.alexnet(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = False


b =[]
for layer in model.classifier.children():
    b.append(layer)
b = b[:-5]

b.append(nn.Linear(4096, 10))
newclassifier = nn.Sequential(*b)
model.classifier = newclassifier

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0002, momentum=0.9)
save_path = './AlexNetc.pth'
bestacc = 0.0
train_steps = len(train_loader)
epochs = 2

#Start the training
print("Starting Training")
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader)
    for i, data in enumerate(train_bar):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.desc = "Train epoch[{}/{}] loss:{:.4f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)


    model.eval()
    acc = 0.0
    with torch.no_grad():
        val_bar = tqdm(val_loader)
        for data in val_bar:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            acc += torch.eq(predicted, labels).sum().item()
    valaccurate = acc / valsize
    print('[epoch %d] training loss: %.4f  value accuracy: %.4f' %
              (epoch + 1, running_loss / train_steps, valaccurate))

    if valaccurate > bestacc:
        bestacc = valaccurate
        torch.save(model.state_dict(), save_path)        
print('Training Finished')

#Starting testing with train partition 
print('Starting to test with train partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(train_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
testaccuracy = acc / 10000
print('Testing accuracy: %.4f' %(testaccuracy))


#Evaluating test partition 
print('Start Testing with valadation partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(val_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
testaccuracy = acc / 10000
print('Testing Accuracy: %.4f' %(testaccuracy))


#Test partition evalation 
print('Testing with Test Partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(test_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
testaccuracy = acc / 10000
print('testing accuracy: %.4f' %(testaccuracy))

Starting Training


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:12<00:00,  1.38it/s]


[epoch 1] training loss: 0.8946  value accuracy: 0.7297


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:13<00:00,  1.37it/s]


[epoch 2] training loss: 0.7141  value accuracy: 0.7449
Training Finished
Starting to test with train partition


...

Testing accuracy: 3.0364
Start Testing with valadation partition


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:12<00:00,  1.37it/s]


Testing Accuracy: 0.7449
Testing with Test Partition


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:08<00:00,  1.46it/s]

testing accuracy: 0.7454


## Part D of Assignment

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model = models.alexnet(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = False


b =[]
for layer in model.classifier.children():
    b.append(layer)
b = b[:-2]

b.append(nn.Linear(4096, 10))
newclassifier = nn.Sequential(*b)
model.classifier = newclassifier

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
save_path = './AlexNetc.pth'
bestacc = 0.0
train_steps = len(train_loader)
epochs = 2

print("Starting Training")
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader)
    for i, data in enumerate(train_bar):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.desc = "train epoch[{}/{}] loss:{:.4f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)


    model.eval()
    acc = 0.0
    with torch.no_grad():
        val_bar = tqdm(val_loader)
        for data in val_bar:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            acc += torch.eq(predicted, labels).sum().item()
    valaccurate = acc / valsize
    print('[epoch %d] Training Loss: %.4f  Accuracy Value: %.4f' %
              (epoch + 1, running_loss / train_steps, valaccurate))

    if valaccurate > bestacc:
        bestacc = valaccurate
        torch.save(model.state_dict(), save_path)

            
print('Training Finished')
print('Testing with training partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(train_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
testaccuracy = acc / 10000
print('test_accuracy: %.4f' %(testaccuracy))


print('Finished Training')
print('Start Testing with validation partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(val_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
testaccuracy = acc / 10000
print('Testing Accuracy: %.4f' %(testaccuracy))


print('Testing with Testing Partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(test_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
testaccuracy = acc / 10000
print('Testing Accuracy: %.4f' %(testaccuracy))

Starting Training


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:16<00:00,  1.30it/s]


[epoch 1] Training Loss: 57.5335  Accuracy Value: 0.6836


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:16<00:00,  1.31it/s]


[epoch 2] Training Loss: 32.0591  Accuracy Value: 0.7203
Training Finished
Testing with training partition


...

test_accuracy: 2.9014
Finished Training
Start Testing with validation partition


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:16<00:00,  1.31it/s]


Testing Accuracy: 0.7203
Testing with Testing Partition


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:11<00:00,  1.40it/s]

Testing Accuracy: 0.7183
